In [184]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    Article
)
from paperview.retrieval import pdf_extraction
import tempfile
from pikepdf import Pdf, PdfImage
from io import StringIO
from PIL import Image

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
# url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
url = 'https://www.biorxiv.org/content/10.1101/456574v1'
# url = 'https://www.biorxiv.org/content/10.1101/2022.12.21.521407v1?rss=1'
article = Article.from_content_page_url(url, prog_bar=True)

100%|██████████| 76/76 [00:54<00:00,  1.39it/s]


In [186]:
import seaborn as sns
import pandas as pd

In [187]:
words = article.data['words']

In [192]:
def find_figure_numbers(images, words, eps=100):
    """For each image in images, uses the page number and bounding box to identify nearby words within distance of eps.
    Searches for cases where the words 'figure', 'fig', etc are present with a number or letter following. 
    Attaches the figure number to the image in the 'figure_number' column.
    """
    for image in images:
        # Find words on the same page
        page_words = words[words['page_number'] == image['page_number']]
        # Find words with bounding box within eps of image
        nearby_words = page_words[
            (page_words['x0'] > image['x0'] - eps) & 
            (page_words['x0'] < image['x1'] + eps) & 
            (page_words['top'] > image['top'] - eps) & 
            (page_words['bottom'] < image['bottom'] + eps)
        ]
        # look for words that contain 'figure' or 'fig'
        figure_words = nearby_words[
            (nearby_words['text'].str.contains('figure', case=False)) |
            (nearby_words['text'].str.contains('fig', case=False))
        ]
        # for each figure word, check if there is a nearby word that is a number or letter
        candidate_figure_numbers = []
        for _, figure_word in figure_words.iterrows():
            figure_number = nearby_words[
                (nearby_words['x0'] > figure_word['x1']) &
                (nearby_words['x0'] < figure_word['x1'] + eps) &
                (nearby_words['top'] > figure_word['top'] - eps) &
                (nearby_words['bottom'] < figure_word['bottom'] + eps)
            ]
            if figure_number.shape[0] > 0:
                if figure_number['text'].str.isnumeric().any():
                    candidate_figure_numbers.append(figure_number['text'].str.extract('(\d+)', expand=False).iloc[0])
                elif figure_number['text'].str.isalpha().any():
                    candidate_figure_numbers.append(figure_number['text'].str.extract('([a-zA-Z]+)', expand=False).iloc[0])
        # choose most common candidate figure number
        if len(candidate_figure_numbers) == 1:
            image['figure_number'] = candidate_figure_numbers[0]
        elif len(candidate_figure_numbers) > 1:
            image['figure_number'] = pd.Series(candidate_figure_numbers).dropna().mode()
        else:
            image['figure_number'] = None
        
    return images

In [ ]:
article.data['images'][0]['image']

In [193]:
find_figure_numbers(article.data['images'], words)

[{'x0': 71.76,
  'y0': 450.96002175,
  'x1': 541.4399925,
  'y1': 720.24000525,
  'width': 469.6799925,
  'height': 269.27998349999996,
  'name': 'Im0',
  'stream': <PDFStream(287): raw=183680, {'BitsPerComponent': 8, 'ColorSpace': /'DeviceRGB', 'Filter': /'DCTDecode', 'Height': 1122, 'Length': 183680, 'Subtype': /'Image', 'Type': /'XObject', 'Width': 1957}>,
  'srcsize': (1957, 1122),
  'imagemask': None,
  'bits': 8,
  'colorspace': [/'DeviceRGB'],
  'object_type': 'image',
  'page_number': 39,
  'top': 71.75999475000003,
  'bottom': 341.03997825,
  'doctop': 30167.75999475,
  'image': <PIL.Image.Image image mode=RGB size=1956x1122>,
  'figure_number': nan},
 {'x0': 71.76,
  'y0': 486.0,
  'x1': 540.24001125,
  'y1': 693.36000825,
  'width': 468.48001124999996,
  'height': 207.36000824999996,
  'name': 'Im0',
  'stream': <PDFStream(288): raw=118579, {'BitsPerComponent': 8, 'ColorSpace': /'DeviceRGB', 'Filter': /'DCTDecode', 'Height': 864, 'Length': 118579, 'Subtype': /'Image', 'Type'

In [ ]:
article.data['images'][0]['image']

,text,x0,x1,top,doctop,bottom,upright,direction
0,bioRxiv,52.54037,78.76437,5.65598,5.65598,13.65598,True,1
1,preprint,80.98837,108.10837,5.65598,5.65598,13.65598,True,1
2,doi:,110.33229,123.22829,5.65598,5.65598,13.65598,True,1
3,https://doi.org/10.1101/2022.07.21.500746;,125.45229,278.45213,5.65598,5.65598,13.65598,True,1
4,this,280.67613,293.12413,5.65598,5.65598,13.65598,True,1
...,...,...,...,...,...,...,...,...
13110,4.0,318.87605,329.99605,21.65606,29195.90606,29.65606,True,1
13111,International,332.22005,376.24405,21.65606,29195.90606,29.65606,True,1
13112,license.,378.46805,405.58789,21.65606,29195.90606,29.65606,True,1
13113,Fig.,34.67124,63.07076,30.07300,29204.32300,46.07300,True,1


In [123]:
pd.DataFrame(article.data['words'])

,text,x0,x1,top,doctop,bottom,upright,direction
0,bioRxiv,52.54037,78.76437,5.65598,29179.90598,13.65598,True,1
1,preprint,80.98837,108.10837,5.65598,29179.90598,13.65598,True,1
2,doi:,110.33229,123.22829,5.65598,29179.90598,13.65598,True,1
3,https://doi.org/10.1101/2022.07.21.500746;,125.45229,278.45213,5.65598,29179.90598,13.65598,True,1
4,this,280.67613,293.12413,5.65598,29179.90598,13.65598,True,1
5,version,295.34813,321.13213,5.65598,29179.90598,13.65598,True,1
6,posted,323.35613,347.37213,5.65598,29179.90598,13.65598,True,1
7,July,349.59613,363.82013,5.65598,29179.90598,13.65598,True,1
8,"22,",366.04413,377.16413,5.65598,29179.90598,13.65598,True,1
9,2022.,379.38813,399.40413,5.65598,29179.90598,13.65598,True,1


In [24]:
import pdfplumber
from tqdm import tqdm

In [105]:
with pdf_extraction.NamedTemporaryPDF(article.article_detail.pdf_url) as pdf_path:
    pdf = pdfplumber.open(pdf_path)
    images = []
    texts = []
    words = []
    tables = []
    for page in tqdm(pdf.pages):
        for image in page.images:
            image_bbox = (image['x0'], image['top'], image['x1'], image['bottom'])
            image['image'] = page.crop(image_bbox).to_image(resolution=300).annotated
            images.append(image)
            
        texts += page.extract_text()
        
        words += page.extract_words()
        
        tables += page.extract_tables()

100%|██████████| 54/54 [00:24<00:00,  2.22it/s]


In [107]:
def identify_images_to_vertically_splice(image_list):
    """Identifies images to vertically splice. Candidate images will be on the same page, have the same width, and the top of one image will be the bottom of the other image. Works for N images"""
    candidates = []
    for i, image1 in enumerate(image_list):
        for j, image2 in enumerate(image_list):
            if i == j:
                continue
            if image1['page_number'] == image2['page_number']:
                if abs(image1['width'] - image2['width']) < 1:
                    if abs(image1['top'] - image2['bottom']) < 1:
                        candidates.append((image2, image1))
                        
    return candidates

def identify_images_to_horizontally_splice(image_list):
    """Identifies images to horizontally splice. Candidate images will be on the same page, have the same height, and the left of one image will be the right of the other image. Works for N images"""
    candidates = []
    for i, image1 in enumerate(image_list):
        for j, image2 in enumerate(image_list):
            if i == j:
                continue
            if image1['page_number'] == image2['page_number']:
                if abs(image1['height'] - image2['height']) < 1:
                    if abs(image1['x1'] - image2['x0']) < 1:
                        candidates.append((image1, image2))
    return candidates


def vertically_splice_image_pair(top_image, bottom_image):
    """Vertically splices two images. Returns a single dictionary for the new image"""
    new_image = top_image.copy()
    new_image['y0'] = bottom_image['y0']
    new_image['y1'] = top_image['y1']
    new_image['height'] = new_image['height'] + new_image['height']
    new_image['top'] = top_image['top']
    new_image['bottom'] = bottom_image['bottom']
    new_image['doctop'] = bottom_image['doctop']
    
    # get height and width from images directly since the values in the dictionary are in a different coordinate frame
    top_image_height = top_image['image'].height
    top_image_width = top_image['image'].width
    bottom_image_height = bottom_image['image'].height
    bottom_image_width = bottom_image['image'].width
    
    # create new image and paste the images
    new_image['image'] = Image.new('RGB', (top_image_width, top_image_height + bottom_image_height))
    new_image['image'].paste(top_image['image'], (0, 0))
    new_image['image'].paste(bottom_image['image'], (0, top_image_height))
    
    return new_image

def horizontally_splice_image_pair(left_image, right_image):
    """Horizontally splices two images. Returns a single dictionary for the new image"""
    new_image = left_image.copy()
    new_image['x0'] = left_image['x0']
    new_image['x1'] = right_image['x1']
    new_image['width'] = left_image['width'] + right_image['width']
    
    
    # get height and width from images directly since the values in the dictionary are in a different coordinate frame
    left_image_height = left_image['image'].height
    left_image_width = left_image['image'].width
    right_image_height = right_image['image'].height
    right_image_width = right_image['image'].width
    
    # create new image and paste the images
    new_image['image'] = Image.new('RGB', (left_image_width + right_image_width, left_image_height))
    new_image['image'].paste(left_image['image'], (0, 0))
    new_image['image'].paste(right_image['image'], (left_image_width, 0))
    
    return new_image



def splice_images(image_list):
    """Identifies sets of images to vertically splice and then splices them. 
    For cases where images are split into N pieces, it iteratively merges them until there is one image"""
    new_image_list = image_list.copy()
    candidates = identify_images_to_vertically_splice(new_image_list)
    while len(candidates) > 0:
        candidate = candidates[0]
        new_image = vertically_splice_image_pair(candidate[0], candidate[1])
        new_image_list.append(new_image)
        if candidate[0] in new_image_list:
            new_image_list.remove(candidate[0])
        if candidate[1] in new_image_list:
            new_image_list.remove(candidate[1])
        candidates = identify_images_to_vertically_splice(new_image_list)
        
    candidates = identify_images_to_horizontally_splice(new_image_list)
    while len(candidates) > 0:
        candidate = candidates[0]
        new_image = horizontally_splice_image_pair(candidate[0], candidate[1])
        new_image_list.append(new_image)
        if candidate[0] in new_image_list:
            new_image_list.remove(candidate[0])
        if candidate[1] in new_image_list:
            new_image_list.remove(candidate[1])
        candidates = identify_images_to_horizontally_splice(new_image_list)
    return new_image_list

    
    

In [108]:
new_images = splice_images(images)

In [112]:
tables[0]

[['To \\ From', 'E1', 'E2', 'E3', 'PV1', 'PV2', 'SOM1', 'SOM2'],
 ['E1 (L2/3)',
  '0.1600',
  '0.0105',
  '0.1400',
  '0.3305',
  '0.0500',
  '0.3370',
  '0.0845'],
 ['E2 (L5/6)',
  '0.0415',
  '0.0503',
  '0.0680',
  '0.0395',
  '0.1713',
  '0.0118',
  '0.1192'],
 ['E3 (L4)',
  '0.0160',
  '0.0035',
  '0.2430',
  '0.2435',
  '0.0500',
  '0.2005',
  '0.0280'],
 ['PV1 (L2/3/4)',
  '0.2390',
  '0.0250',
  '0.2650',
  '0.2505',
  '0.0340',
  '0.4535',
  '0.0075'],
 ['PV2 (L5/6)',
  '0.0405',
  '0.0670',
  '0.0505',
  '0.0410',
  '0.1202',
  '0.0075',
  '0.2368'],
 ['SOM1(L2/3/4)',
  '0.1325',
  '0.0250',
  '0.3355',
  '0.0400',
  '0.0075',
  '0.0660',
  '0.0057'],
 ['SOM2 (L5/6)',
  '0.0510',
  '0.0508',
  '0.0640',
  '0.0075',
  '0.0425',
  '0',
  '0.0325']]